In [170]:
import numpy as np
import pandas as pd
import itertools
import csv

In [164]:
i = ['queso','leche','pollo','ternera','refresco','revista']
t = []

t1 = set(['ternera','leche','pollo'])
t2 = set(['ternera','queso','pollo','leche'])
t3 = set(['queso','pollo','leche','refresco'])
t4 = set(['ternera','queso','pollo','leche'])
t5 = set(['ternera','pollo','queso','revista','leche'])
t6 = set(['revista','leche','pollo'])
t7 = set(['revista','leche','pollo'])
t = [t1,t2,t3,t4,t5,t6,t7]
t


[{'leche', 'pollo', 'ternera'},
 {'leche', 'pollo', 'queso', 'ternera'},
 {'leche', 'pollo', 'queso', 'refresco'},
 {'leche', 'pollo', 'queso', 'ternera'},
 {'leche', 'pollo', 'queso', 'revista', 'ternera'},
 {'leche', 'pollo', 'revista'},
 {'leche', 'pollo', 'revista'}]

### Obtenemos todos los subconjuntos X ( antecendentes ) , Y (consecuentes)


In [165]:
Y = {}
for l in range(1,len(i)):
    for subset in itertools.combinations(i, l):
        Y[subset] = list(set(i) - set(subset))
Y

{('queso',): ['pollo', 'refresco', 'ternera', 'revista', 'leche'],
 ('leche',): ['queso', 'pollo', 'refresco', 'ternera', 'revista'],
 ('pollo',): ['queso', 'refresco', 'ternera', 'revista', 'leche'],
 ('ternera',): ['queso', 'pollo', 'refresco', 'revista', 'leche'],
 ('refresco',): ['queso', 'pollo', 'ternera', 'revista', 'leche'],
 ('revista',): ['queso', 'pollo', 'refresco', 'ternera', 'leche'],
 ('queso', 'leche'): ['ternera', 'revista', 'pollo', 'refresco'],
 ('queso', 'pollo'): ['ternera', 'revista', 'refresco', 'leche'],
 ('queso', 'ternera'): ['revista', 'pollo', 'refresco', 'leche'],
 ('queso', 'refresco'): ['ternera', 'revista', 'pollo', 'leche'],
 ('queso', 'revista'): ['ternera', 'pollo', 'refresco', 'leche'],
 ('leche', 'pollo'): ['ternera', 'queso', 'revista', 'refresco'],
 ('leche', 'ternera'): ['revista', 'queso', 'pollo', 'refresco'],
 ('leche', 'refresco'): ['ternera', 'queso', 'pollo', 'revista'],
 ('leche', 'revista'): ['ternera', 'queso', 'pollo', 'refresco'],
 ('p

### Para cada antecedente y cada consecuente de este ( ahora llamado regla ), obtenemos todas las ocurrencias en las transacciones

In [166]:
rules = {}
for X in Y.keys():
    for item in Y[X]:
        for transaction in t:
            if set(list(X)+[item]).issubset(set(transaction)):
                rules[X,item] = rules.get((X,item),0) +1
rules

{(('queso',), 'pollo'): 4,
 (('queso',), 'refresco'): 1,
 (('queso',), 'ternera'): 3,
 (('queso',), 'revista'): 1,
 (('queso',), 'leche'): 4,
 (('leche',), 'queso'): 4,
 (('leche',), 'pollo'): 7,
 (('leche',), 'refresco'): 1,
 (('leche',), 'ternera'): 4,
 (('leche',), 'revista'): 3,
 (('pollo',), 'queso'): 4,
 (('pollo',), 'refresco'): 1,
 (('pollo',), 'ternera'): 4,
 (('pollo',), 'revista'): 3,
 (('pollo',), 'leche'): 7,
 (('ternera',), 'queso'): 3,
 (('ternera',), 'pollo'): 4,
 (('ternera',), 'revista'): 1,
 (('ternera',), 'leche'): 4,
 (('refresco',), 'queso'): 1,
 (('refresco',), 'pollo'): 1,
 (('refresco',), 'leche'): 1,
 (('revista',), 'queso'): 1,
 (('revista',), 'pollo'): 3,
 (('revista',), 'ternera'): 1,
 (('revista',), 'leche'): 3,
 (('queso', 'leche'), 'ternera'): 3,
 (('queso', 'leche'), 'revista'): 1,
 (('queso', 'leche'), 'pollo'): 4,
 (('queso', 'leche'), 'refresco'): 1,
 (('queso', 'pollo'), 'ternera'): 3,
 (('queso', 'pollo'), 'revista'): 1,
 (('queso', 'pollo'), 'refr

### Obtenemos el soporte de cada regla

In [167]:
soporte = {}
for rule in rules.keys():
    soporte[rule] = rules[rule]/len(t)

### Funcion que realiza lo anterior

In [202]:
def ConjuntosFrecuentesFB(i,t,umbral):
    Y = {}
    rules = {}
    soporte = {}
    
    #Todos los antecedentes y sus posibles consecuentes
    for l in range(1,len(i)):
        for subset in itertools.combinations(i, l):
            Y[subset] = list(set(i) - set(subset))
    
    #Creacion de reglas y obtencion de sus ocurrencias
    for X in Y.keys():
        for item in Y[X]:
            for transaction in t:
                if set(list(X)+[item]).issubset(set(transaction)):
                    rules[X,item] = rules.get((X,item),0) +1
    #Obtencion del soporte
    for i in rules.keys():
        soporte[i] = rules[i]/len(t)
        if soporte[i] >= umbral:
            print(i,soporte[i])
    return rules,soporte
rules,soporte = ConjuntosFrecuentesFB(i,t,.5)

(('Jelly',), 'Fruit') 0.5
(('Soda',), 'Milk') 0.5
(('Soda',), 'Fruit') 0.6
(('Milk',), 'Soda') 0.5
(('Milk',), 'Fruit') 0.6
(('Fruit',), 'Jelly') 0.5
(('Fruit',), 'Soda') 0.6
(('Fruit',), 'Milk') 0.6
(('Soda', 'Milk'), 'Fruit') 0.5
(('Soda', 'Fruit'), 'Milk') 0.5
(('Milk', 'Fruit'), 'Soda') 0.5


### Lectura de archivo transactions.csv


In [206]:
def read_csv(file):
    i = []
    t = {}
    transactions = []
    with open('transactions.csv', mode='r') as csv_file:
        csv_reader = csv.DictReader(csv_file)
        for row in csv_reader:
            if row['Customer'] not in t.keys():
                t[row['Customer']] = []
            t[row['Customer']].append(row['Product'])
            i.append(row['Product'])
    i = list(set(i))
    for key in t.keys():
        transactions.append(set(t[key]))
    return i,transactions

In [207]:
i,t = read_csv('transactions.csv')
rules, support = ConjuntosFrecuentesFB(i,t,.6)

(('Soda',), 'Fruit') 0.6
(('Milk',), 'Fruit') 0.6
(('Fruit',), 'Soda') 0.6
(('Fruit',), 'Milk') 0.6
